In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os
os.chdir('/home/ubuntu/fastai/courses/dl1/')

In [4]:
import torch
import pandas as pd
import numpy as np
import PIL
from IPython.lib.display import FileLink

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
torch.cuda.set_device(0)

In [6]:
from fastai.dataset import get_cv_idxs
from fastai.transforms import tfms_from_model, transforms_side_on
from fastai.conv_learner import ImageClassifierData, ConvLearner
from fastai.torch_imports import resnext101_64

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
PATH = "data/ass1/"
sz = 224
arch = resnext101_64
bs = 58

In [8]:
label_csv = f'{PATH}train.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

In [9]:
n

7349

In [10]:
len(val_idxs)

1469

In [11]:
!ls {PATH}

sample_submission.csv  subm  test  tmp	train  train.csv


In [12]:
label_df = pd.read_csv(label_csv)

In [13]:
label_df.head()

id           breed
0  b0234bf14351b079397d8685b2907e8d        keeshond
1  378e20d6c6c2f08fb0d9dce02b22caf4    newfoundland
2  b900795a1441629688fcd8935def1e4e  great_pyrenees
3  a8d369de5c7ef297d19316a19ab24501  great_pyrenees
4  8bff692933224c78840b10d3c1c22a16      leonberger

In [14]:
label_df.pivot_table(index="breed", aggfunc=len).sort_values('id', ascending=False)

id
breed                          
yorkshire_terrier           200
samoyed                     200
russian_blue                200
ragdoll                     200
pug                         200
pomeranian                  200
persian                     200
miniature_pinscher          200
maine_coon                  200
leonberger                  200
american_bulldog            200
japanese_chin               200
havanese                    200
great_pyrenees              200
german_shorthaired          200
english_setter              200
shiba_inu                   200
chihuahua                   200
british_shorthair           200
sphynx                      200
wheaten_terrier             200
birman                      200
bengal                      200
beagle                      200
basset_hound                200
american_pit_bull_terrier   200
saint_bernard               200
siamese                     199
scottish_terrier            199
keeshond                    199
boxer                       199
abyssinian                  198
newfoundland                196
english_cocker_spaniel      196
egyptian_mau                190
staffordshire_bull_terrier  189
bombay                      184

In [15]:
tfms =tfms_from_model(arch,sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}train.csv', test_name='test', 
                                    val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

In [16]:
fn = PATH + data.trn_ds.fnames[0]
fn

'data/ass1/train/378e20d6c6c2f08fb0d9dce02b22caf4.jpg'

In [17]:
img = PIL.Image.open(fn)
img

In [18]:
def get_data(sz, bs): # sz: image size, bs: batch size
    tmfs = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}train.csv', test_name='test',
                                       val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/13
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/37
    return data if sz > 300 else data.resize(340, 'tmp') # Reading the jpgs and resizing is slow for big images, so resizing them all to 340 first saves time

In [19]:
data = get_data(sz, bs)

In [20]:
learn = ConvLearner.pretrained(arch,data, precompute= True)
learn.fit(1e-2, 5)

epoch      trn_loss   val_loss   accuracy                   
    0      0.549192   0.213962   0.936011  
    1      0.279675   0.174653   0.94418                    
    2      0.19287    0.166426   0.945541                   
    3      0.144814   0.152399   0.946222                   
    4      0.129931   0.152458   0.947583                   



[array([0.15246]), 0.9475833931855396]

# Create submission

In [21]:
data.classes

['abyssinian',
 'american_bulldog',
 'american_pit_bull_terrier',
 'basset_hound',
 'beagle',
 'bengal',
 'birman',
 'bombay',
 'boxer',
 'british_shorthair',
 'chihuahua',
 'egyptian_mau',
 'english_cocker_spaniel',
 'english_setter',
 'german_shorthaired',
 'great_pyrenees',
 'havanese',
 'japanese_chin',
 'keeshond',
 'leonberger',
 'maine_coon',
 'miniature_pinscher',
 'newfoundland',
 'persian',
 'pomeranian',
 'pug',
 'ragdoll',
 'russian_blue',
 'saint_bernard',
 'samoyed',
 'scottish_terrier',
 'shiba_inu',
 'siamese',
 'sphynx',
 'staffordshire_bull_terrier',
 'wheaten_terrier',
 'yorkshire_terrier']

In [22]:
log_preds, y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds),0)

In [23]:
probs.shape

(6849, 37)

In [24]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [25]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [26]:
df.head()

id    abyssinian  american_bulldog  \
0  71ee0ff967315f876e58845a88c73d2b  3.151136e-04      1.264985e-05   
1  a2e38810773b5f34401c8e4307526972  5.379618e-05      1.815226e-05   
2  1ea3b76135bb4a589027d6243075a936  3.587907e-07      6.807211e-05   
3  9e9505cf0106e91ee31fe671c83e5ee3  1.769889e-06      6.176860e-08   
4  e6da34ba138d28581b60b2913c264f1c  3.236164e-04      6.559042e-05   

   american_pit_bull_terrier  basset_hound        beagle        bengal  \
0               5.102784e-06  1.472771e-05  3.129652e-05  1.242214e-06   
1               2.207322e-05  3.256613e-06  2.394736e-05  1.008201e-04   
2               1.327684e-05  3.114028e-05  3.538895e-06  1.016468e-06   
3               6.718216e-07  5.086937e-07  1.097626e-07  1.647199e-07   
4               7.015723e-04  5.349985e-04  1.603537e-04  8.184506e-04   

         birman        bombay     boxer        ...          russian_blue  \
0  3.927581e-03  8.162092e-06  0.000028        ...          5.162262e-05   
1  9.652620e-05  3.119927e-04  0.000009        ...          9.106909e-05   
2  3.267319e-05  9.243655e-07  0.000004        ...          1.206151e-06   
3  3.184667e-08  2.536002e-07  0.000001        ...          6.803835e-07   
4  1.299902e-02  5.994326e-03  0.000706        ...          6.331850e-04   

   saint_bernard       samoyed  scottish_terrier  shiba_inu       siamese  \
0       0.000215  1.183100e-05      5.187072e-06   0.000025  5.514256e-05   
1       0.000035  9.762161e-06      5.190922e-06   0.000013  4.115590e-06   
2       0.000205  4.226494e-04      2.519993e-06   0.000003  7.642702e-06   
3       0.000036  6.633212e-07      6.994825e-07   0.000001  3.784114e-07   
4       0.000332  4.007078e-04      1.813822e-03   0.000177  4.867366e-04   

         sphynx  staffordshire_bull_terrier  wheaten_terrier  \
0  9.481529e-06                2.812272e-06     4.882705e-05   
1  3.009580e-06                3.577812e-06     4.297332e-06   
2  2.077123e-06                4.208559e-06     2.277269e-04   
3  5.367798e-08                2.507447e-07     5.120961e-07   
4  6.311195e-04                3.502473e-04     7.808969e-04   

   yorkshire_terrier  
0       1.025738e-05  
1       3.167487e-06  
2       5.965380e-07  
3       7.458874e-07  
4       1.089843e-03  

[5 rows x 38 columns]

In [27]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [28]:
FileLink(f'{SUBM}subm.gz')

/home/ubuntu/fastai/courses/dl1/data/ass1/subm/subm.gz

In [36]:
ls

subm.gz


In [37]:
!kaggle competitions submit -c oxford-iiit-pet-dataset -f subm.gz -m "My submission"

Successfully submitted to Spaceport.AI Assignment 1: Dog and Cat Breeds